# Preprocesamiento con scikit-learn

### SimpleImputer

In [3]:
from sklearn.impute import SimpleImputer
import pandas as pd

# Datos con valores faltantes
x = pd.DataFrame({
    'A': [1,3,5],
    'B': [2,None,4],
    'C': [None,6,9]
})

print(x)

   A    B    C
0  1  2.0  NaN
1  3  NaN  6.0
2  5  4.0  9.0


In [4]:
# Crear un imputador con la estrategia de la media
imputer = SimpleImputer(strategy='mean')

# Ajustar y transformar los datos
x_imputer = imputer.fit_transform(x)

print(x_imputer)

[[1.  2.  7.5]
 [3.  3.  6. ]
 [5.  4.  9. ]]


In [5]:
# Convertir el resultado a un DataFrame de pandas
x_imputed_df = pd.DataFrame(x_imputer, columns=x.columns)

print("nDatos imputados:")
print(x_imputed_df)

nDatos imputados:
     A    B    C
0  1.0  2.0  7.5
1  3.0  3.0  6.0
2  5.0  4.0  9.0


### OneHotEncoder

In [6]:
from sklearn.preprocessing import OneHotEncoder

# Datos categóricos
X = [['rojo'], ['verde'], ['azul'], ['verde'], ['rojo']]

# Crear un codificador One-Hot
encoder = OneHotEncoder()

# Ajustar y transformar los datos
X_encoded = encoder.fit_transform(X)

print(X_encoded.toarray())

[[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]


### LabelEncoder

In [7]:
from sklearn.preprocessing import LabelEncoder

# Datos categóricos
y = ['rojo', 'verde', 'azul', 'verde', 'rojo']

# Crear un codificador de etiquetas
encoder = LabelEncoder()

# Ajustar y transformar los datos
y_encoded = encoder.fit_transform(y)

print(y_encoded)

[1 2 0 2 1]


### Ejemplo

In [8]:
from sklearn.datasets import fetch_openml
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Cargar el conjunto de datos
X, y = fetch_openml('tic-tac-toe', version=1, return_X_y=True)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear un imputador para manejar valores faltantes
imputer = SimpleImputer(strategy='most_frequent')

# Crear un codificador One-Hot para variables categóricas
encoder = OneHotEncoder()

# Preprocesar los datos de entrenamiento
X_train_imputed = imputer.fit_transform(X_train)
X_train_encoded = encoder.fit_transform(X_train_imputed)

# Preprocesar los datos de prueba
X_test_imputed = imputer.transform(X_test)
X_test_encoded = encoder.transform(X_test_imputed)

# Codificar las etiquetas de destino
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Crear y entrenar un clasificador de árboles de decisión
classifier = DecisionTreeClassifier()
classifier.fit(X_train_encoded, y_train_encoded)

# Hacer predicciones en el conjunto de prueba
y_pred = classifier.predict(X_test_encoded)

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test_encoded, y_pred)
print(f"Precisión del modelo: {accuracy}")

Precisión del modelo: 0.96875


### ColumnTransformer

In [1]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Datos de ejemplo
X = [[1, 'rojo', 10],
     [2, 'azul', None],
     [None, 'verde', 12],
     [4, 'rojo', 8]]

# Definir las columnas numéricas y categóricas
numeric_features = [0, 2]
categorical_features = [1]

# Crear transformadores para cada subconjunto de columnas
numeric_transformer = SimpleImputer(strategy='mean')
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Crear un ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Aplicar el preprocesamiento
X_transformed = preprocessor.fit_transform(X)

print(X_transformed)

[[ 1.         10.          0.          1.          0.        ]
 [ 2.         10.          1.          0.          0.        ]
 [ 2.33333333 12.          0.          0.          1.        ]
 [ 4.          8.          0.          1.          0.        ]]


Interpretación del resultado
Imputación de valores faltantes:

El valor faltante en la columna 0 (para la tercera fila) se reemplaza con la media de la columna 0, que es aproximadamente 2.333.
El valor faltante en la columna 2 (para la segunda fila) se reemplaza con la media de la columna 2, que es 10.
Codificación One-Hot:

La columna 1 original con los valores 'rojo', 'azul', y 'verde' se transforma en tres columnas binarias:
La primera columna binaria representa 'rojo', la segunda 'azul', y la tercera 'verde'.
Ahora cada fila contiene valores codificados en formato numérico, listos para ser usados en un modelo de machine learning.

Este proceso de preprocesamiento garantiza que el modelo reciba datos numéricos y que no haya valores faltantes, haciendo que el conjunto de datos sea adecuado para su uso en modelos de machine learning.

### Pipeline

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# Crear un Pipeline
pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression())
])

# Ajustar el Pipeline a los datos de entrenamiento
pipeline.fit(X_train, y_train)

# Hacer predicciones utilizando el Pipeline
y_pred = pipeline.predict(X_test)

ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: 'o'

#### Combinando ColumnTransformer y Pipeline

In [17]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml

# Cargar el conjunto de datos
X, y = fetch_openml('titanic', version=1, as_frame=True, return_X_y=True)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir las columnas numéricas y categóricas
numeric_features = ['age', 'fare']
categorical_features = ['embarked', 'sex', 'pclass']

# Crear transformadores para cada subconjunto de columnas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Crear un ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# Crear un Pipeline completo
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

# Ajustar el Pipeline a los datos de entrenamiento
pipeline.fit(X_train, y_train)

# Hacer predicciones utilizando el Pipeline
y_pred = pipeline.predict(X_test)

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo: {accuracy:.2f}")

Precisión del modelo: 0.77
